In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [2]:
def dec2bin(x, n=7):
    """
    creating a binary list of integer non-negative x
    """
    u = 2 ** n - 1
    x = int(x)
    assert x >= 0, 'Input value x must be non-negative'
    assert x <= u, f'Input value x with n = {n} must be less than {u}'
    
    y = []
    if x == 0:
        for i in range(n):
            y.append(0)
    else:
        while x != 1:
            y.append(x % 2)
            x = x // 2
        y.append(x)
        delta = n - len(y)
        for i in range(delta):
            y.append(0)
        y.reverse()
    return y

In [3]:
def get_all_X(n_inputs):
    """
    for given number of variables returns
    all binary combinations of these variables
    """
    X = []
    for i in range(2**n_inputs):
        X += [dec2bin(i, n=n_inputs)]
    X = np.array(X)
    return X

In [4]:
def random_boolfunc(n_inputs):
    """
    for given number of variables returns
    the truth table, where output is random binary vector
    """
    X = get_all_X(n_inputs)
    y = np.random.randint(0, 2, size=(2**n_inputs, 1))
    return X, y
   
def _and(n_inputs=2):
    """
    for given number of variables returns
    the truth table of AND logical gate
    """
    X = get_all_X(n_inputs)
    y = np.append(
        np.zeros(shape=(2 ** n_inputs - 1, 1), dtype=int),
        np.array([[1]]),
        axis=0
    )
    return X, y
    
def _or(n_inputs=2):
    """
    for given number of variables returns
    the truth table of OR logical gate
    """
    X = get_all_X(n_inputs)
    y = np.append(
        np.array([[0]]),
        np.ones(shape=(2 ** n_inputs - 1, 1), dtype=int),
        axis=0
    )
    return X, y

def _xor(n_inputs=2):
    """
    for given number of variables returns
    the truth table of XOR logical gate
    """
    X = get_all_X(n_inputs)
    y = (np.sum(X, axis=1) % 2).reshape(-1, 1)
    return X, y

def to_dataframe(X, y):
    """
    for the truth table in form of two arrays 
    X [2 ** n_inputs, n_inputs] and y [2 ** n_inputs, 1]
    combine it to the form of Pandas DataFrame
    """
    data=np.concatenate((X, y), axis=1)
    n_inputs = X.shape[1]
    return pd.DataFrame(data=data, columns=[f'x{i}' for i in range(n_inputs, 0, -1)] + ['y'])

In [61]:
n = 3

In [68]:
%%time
X, y = random_boolfunc(n_inputs=n)
to_dataframe(X, y)

Wall time: 0 ns


,x3,x2,x1,y
0,0,0,0,0
1,0,0,1,0
2,0,1,0,1
3,0,1,1,1
4,1,0,0,1
5,1,0,1,1
6,1,1,0,0
7,1,1,1,1


In [69]:
X, y = torch.Tensor(X), torch.Tensor(y)

In [70]:
class Neuron(nn.Module):
    def __init__(self, n_inputs, activation=nn.Sigmoid()):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, 1)
        #self.fc2 = nn.Linear(2,1)
        self.activation = activation
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        #x = self.fc2(x)
        #x = self.activation(x)
        return x

In [71]:
model = Neuron(n_inputs=n)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.MSELoss()
n_epochs = 5000
for i in range(1, n_epochs+1):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f'epoch {i}: loss {loss:.2f}')

epoch 100: loss 0.60
epoch 200: loss 0.57
epoch 300: loss 0.56
epoch 400: loss 0.55
epoch 500: loss 0.55
epoch 600: loss 0.55
epoch 700: loss 0.55
epoch 800: loss 0.55
epoch 900: loss 0.55
epoch 1000: loss 0.55
epoch 1100: loss 0.55
epoch 1200: loss 0.55
epoch 1300: loss 0.55
epoch 1400: loss 0.55
epoch 1500: loss 0.55
epoch 1600: loss 0.55
epoch 1700: loss 0.55
epoch 1800: loss 0.55
epoch 1900: loss 0.55
epoch 2000: loss 0.55
epoch 2100: loss 0.55
epoch 2200: loss 0.55
epoch 2300: loss 0.55
epoch 2400: loss 0.55
epoch 2500: loss 0.55
epoch 2600: loss 0.55
epoch 2700: loss 0.55
epoch 2800: loss 0.55
epoch 2900: loss 0.55
epoch 3000: loss 0.55
epoch 3100: loss 0.55
epoch 3200: loss 0.55
epoch 3300: loss 0.55
epoch 3400: loss 0.55
epoch 3500: loss 0.55
epoch 3600: loss 0.55
epoch 3700: loss 0.55
epoch 3800: loss 0.55
epoch 3900: loss 0.55
epoch 4000: loss 0.55
epoch 4100: loss 0.55
epoch 4200: loss 0.55
epoch 4300: loss 0.55
epoch 4400: loss 0.55
epoch 4500: loss 0.55
epoch 4600: loss 0.

In [88]:
output

tensor([[0.2151],
        [0.5000],
        [0.5000],
        [0.7849],
        [0.5000],
        [0.7849],
        [0.7849],
        [0.9302]], grad_fn=<SigmoidBackward0>)

In [90]:
torch.heaviside(output - 0.5, torch.tensor([[1.]]))

tensor([[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<NotImplemented>)

In [91]:
y

tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]])

In [109]:
X

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 1.],
        [1., 0., 0.],
        [1., 0., 1.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [98]:
mask = (torch.heaviside(output - 0.5, torch.tensor([[1.]])) != y)[:,0]
mask

tensor([False,  True, False, False, False, False,  True, False])

In [103]:
X_new = X[mask]
y_new = y[mask]
X_new, y_new

(tensor([[0., 0., 1.],
         [1., 1., 0.]]),
 tensor([[0.],
         [0.]]))

In [104]:
model2 = Neuron(n_inputs=n)
optimizer = torch.optim.Adam(model2.parameters(), lr = 0.1)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.MSELoss()
n_epochs = 5000
for i in range(1, n_epochs+1):
    optimizer.zero_grad()
    pred2 = model2(X_new)
    loss = criterion(pred2, y_new)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f'epoch {i}: loss {loss:.2f}')

epoch 100: loss 0.00
epoch 200: loss 0.00
epoch 300: loss 0.00
epoch 400: loss 0.00
epoch 500: loss 0.00
epoch 600: loss 0.00
epoch 700: loss 0.00
epoch 800: loss 0.00
epoch 900: loss 0.00
epoch 1000: loss 0.00
epoch 1100: loss 0.00
epoch 1200: loss 0.00
epoch 1300: loss 0.00
epoch 1400: loss 0.00
epoch 1500: loss 0.00
epoch 1600: loss 0.00
epoch 1700: loss 0.00
epoch 1800: loss 0.00
epoch 1900: loss 0.00
epoch 2000: loss 0.00
epoch 2100: loss 0.00
epoch 2200: loss 0.00
epoch 2300: loss 0.00
epoch 2400: loss 0.00
epoch 2500: loss 0.00
epoch 2600: loss 0.00
epoch 2700: loss 0.00
epoch 2800: loss 0.00
epoch 2900: loss 0.00
epoch 3000: loss 0.00
epoch 3100: loss 0.00
epoch 3200: loss 0.00
epoch 3300: loss 0.00
epoch 3400: loss 0.00
epoch 3500: loss 0.00
epoch 3600: loss 0.00
epoch 3700: loss 0.00
epoch 3800: loss 0.00
epoch 3900: loss 0.00
epoch 4000: loss 0.00
epoch 4100: loss 0.00
epoch 4200: loss 0.00
epoch 4300: loss 0.00
epoch 4400: loss 0.00
epoch 4500: loss 0.00
epoch 4600: loss 0.

In [105]:
pred2, y_new

(tensor([[6.7834e-06],
         [3.4405e-06]], grad_fn=<SigmoidBackward0>),
 tensor([[0.],
         [0.]]))

In [110]:
torch.heaviside(pred2 - 0.5, torch.tensor([[1.]]))

tensor([[0.],
        [0.]], grad_fn=<NotImplemented>)

In [111]:
y_new

tensor([[0.],
        [0.]])

In [204]:
class BigModel(nn.Module):
    def __init__(self, base_models: list):
        super().__init__()
        self.base_models = base_models
        
        for model in self.base_models:
            for param in model.parameters():
                param.requires_grad = False
        
        self.n_hidden = len(base_models)
        self.fc = nn.Linear(self.n_hidden, 1)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        hidden_out = torch.empty(x.size()[0],self.n_hidden)
        for n, base_model in enumerate(self.base_models):
            tmp = base_model(x)[:,0]
            hidden_out[:,n] = tmp
        
        out = self.activation(self.fc(hidden_out))
        
        return out

In [205]:
big_model = BigModel([model, model2])

In [206]:
model(X)[:,0].size()

torch.Size([8])

In [207]:
big_model(X)

tensor([[0.4769],
        [0.4545],
        [0.4545],
        [0.4316],
        [0.4545],
        [0.4316],
        [0.4316],
        [0.4200]], grad_fn=<SigmoidBackward0>)

In [208]:
for param in big_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.3256, -0.4611]], requires_grad=True)
Parameter containing:
tensor([-0.0198], requires_grad=True)


In [214]:
optimizer = torch.optim.Adam(big_model.parameters(), lr = 0.01)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.MSELoss()
n_epochs = 5000
for i in range(1, n_epochs+1):
    optimizer.zero_grad()
    pred3 = big_model(X)
    loss = criterion(pred3, y)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f'epoch {i}: loss {loss:.2f}')

epoch 100: loss 0.53
epoch 200: loss 0.53
epoch 300: loss 0.53
epoch 400: loss 0.53
epoch 500: loss 0.53
epoch 600: loss 0.53
epoch 700: loss 0.53
epoch 800: loss 0.53
epoch 900: loss 0.53
epoch 1000: loss 0.53
epoch 1100: loss 0.53
epoch 1200: loss 0.53
epoch 1300: loss 0.53
epoch 1400: loss 0.53
epoch 1500: loss 0.53
epoch 1600: loss 0.53
epoch 1700: loss 0.53
epoch 1800: loss 0.53
epoch 1900: loss 0.53
epoch 2000: loss 0.53
epoch 2100: loss 0.53
epoch 2200: loss 0.53
epoch 2300: loss 0.53
epoch 2400: loss 0.53
epoch 2500: loss 0.53
epoch 2600: loss 0.53
epoch 2700: loss 0.53
epoch 2800: loss 0.53
epoch 2900: loss 0.53
epoch 3000: loss 0.53
epoch 3100: loss 0.53
epoch 3200: loss 0.53
epoch 3300: loss 0.53
epoch 3400: loss 0.53
epoch 3500: loss 0.53
epoch 3600: loss 0.53
epoch 3700: loss 0.53
epoch 3800: loss 0.53
epoch 3900: loss 0.53
epoch 4000: loss 0.53
epoch 4100: loss 0.53
epoch 4200: loss 0.53
epoch 4300: loss 0.53
epoch 4400: loss 0.53
epoch 4500: loss 0.53
epoch 4600: loss 0.

In [215]:
pred3

tensor([[0.0553],
        [0.6107],
        [0.5998],
        [0.7702],
        [0.5934],
        [0.7702],
        [0.7700],
        [0.8313]], grad_fn=<SigmoidBackward0>)

In [216]:
output

tensor([[0.2151],
        [0.5000],
        [0.5000],
        [0.7849],
        [0.5000],
        [0.7849],
        [0.7849],
        [0.9302]], grad_fn=<SigmoidBackward0>)

In [213]:
y

tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]])

In [217]:
model2(X)

tensor([[6.0751e-03],
        [6.7796e-06],
        [1.1657e-04],
        [1.2932e-07],
        [1.8024e-04],
        [1.9996e-07],
        [3.4386e-06],
        [3.8141e-09]])